In [1]:
import evaluate
import transformers
import argparse
import torch
import bitsandbytes
from datasets import Dataset

from datasets import load_from_disk
from numpy import mean
import os
os.chdir('/storagenfs/l.miglior/hlt-project/')

LLAMA_PATH = '/storagenfs/l.miglior/hlt-project/results/sft-eval/checkpoint-7600'
print(LLAMA_PATH)

nf4_config = transformers.BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float16
)

dataset = load_from_disk('dataset/stackoverflow-paired-11k/sft')
bertscore = evaluate.load("bertscore")


/storagenfs/l.miglior/hlt-project/results/sft-eval/checkpoint-7600


In [4]:
print(f'> Loading model from {LLAMA_PATH}')
print("> Loading tokenizer...")
tokenizer = transformers.AutoTokenizer.from_pretrained(LLAMA_PATH, use_fast=True)
print("> Tokenizer loaded.\n> Loading model...")
model = transformers.AutoModelForCausalLM.from_pretrained(LLAMA_PATH, device_map="cuda:1", quantization_config=nf4_config)
print("> Model Loaded.")


> Loading model from /storagenfs/l.miglior/hlt-project/results/sft-eval/checkpoint-7600
> Loading tokenizer...
> Tokenizer loaded.
> Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

> Model Loaded.


In [6]:
pipe = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="cuda:1",
    tokenizer=tokenizer,
)

In [7]:
val_dataset = dataset.train_test_split(test_size=0.05, seed=42)['test']
val_dataset=val_dataset.map(lambda x: {"question": "### Question: " + x["question"] + " ### Answer: "})

In [16]:
pipe("### Question: How do i write hello world in python? ### Answer: ", temperature=0.9, do_sample=True,
        top_k=10,
        temperature=0.9,
        num_return_sequences=1,)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/storagenfs/l.miglior/hlt-project/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': '### Question: How do i write hello world in python? ### Answer: \n nobody can write'}]

In [12]:
torch.cuda.empty_cache()

In [13]:
a=None
for out in pipe(val_dataset["question"], batch_size=4, max_new_tokens=64, temperature=0.9):
    print(out)
    a=out
    break

/storagenfs/l.miglior/hlt-project/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.60 GiB (GPU 1; 15.89 GiB total capacity; 10.34 GiB already allocated; 258.12 MiB free; 14.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
a

[{'generated_text': '### Question: What problem/s does a Rule Engine Algorithm solve?  \n\nWhich are the recommended ones? ### Answer: '}]

In [46]:
example = dataset.select([3400])
example['question']

['I want to check if a element is being hovered over. I get this error: \n\n```\nSyntax error, unrecognized expression: unsupported pseudo: hover\n\n```\n\nwhen I use this code:\n\n```\n $(\'.class\').blur(function(){\n    if(!$(this).is(\':hover\')){\n       //element not being hovered over\n    }\n });\n\n```\n\ni also tried this:\n\n```\n $(\'.class\').blur(function(){\n    if($(this+":hover").length === 0){ \n       //element not being hovered over\n    }\n });\n\n```\n\nthis also does not work. Is there any other way of doing this. Thanks.']

In [49]:
q = example["question"]
a = example["response_j"]
print('Evaluating question: ',  f'{q[:128]}', '...')
sequences = prediction = pipeline(
    f'### Question: {q} ### Answer:',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=320,
    temperature=0.9,
)
prediction = str([s['generated_text'] for s in sequences])
prediction = prediction.split('### Answer:')[1]

# remove \\n and \\t and replace with space
prediction = prediction.replace('\\n', ' ').replace('\\t', ' ')
# replace "\\\" with empty string
prediction = prediction.replace('\\\"', '')
prediction

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Evaluating question:  ['I want to check if a element is being hovered over. I get this error: \n\n```\nSyntax error, unrecognized expression: unsupported pseudo: hover\n\n```\n\nwhen I use this code:\n\n```\n $(\'.class\').blur(function(){\n    if(!$(this).is(\':hover\')){\n       //element not being hovered over\n    }\n });\n\n```\n\ni also tried this:\n\n```\n $(\'.class\').blur(function(){\n    if($(this+":hover").length === 0){ \n       //element not being hovered over\n    }\n });\n\n```\n\nthis also does not work. Is there any other way of doing this. Thanks.'] ...


" This is a jQuery issue.  ``` $(\\\\\\'.class\\\\\\').blur(function() {   if(!$(this).is(\\\\\\':hover\\\\\\')) {     // element not being hovered over   } });  ```  should work. But the problem may be related to jQuery, because I tried the same code on [jsFiddle](http://jsfiddle.net/mjr39/3/), and it works perfectly.  I suggest that you try this:  ``` $(\\\\\\'.class\\\\\\').hover(function() {   $(this).removeClass(\\'class\\'); }).blur(function() {   if(!$(this).is(\\\\\\':hover\\\\\\')) {     // element not being hovered over   } });  ```  You can read more about [jQuery `.hover()`](http://api.jquery.com/hover/) and [jQuery `.blur()`](http://api.jquery.com/blur/) here.   "

In [51]:
bertscore.compute(predictions=[prediction], references=example['response_j'], lang='en', model_type='bert-base-uncased', device='cuda:0')

{'precision': [0.5083277821540833],
 'recall': [0.7286123037338257],
 'f1': [0.5988549590110779],
 'hashcode': 'bert-base-uncased_L9_no-idf_version=0.3.12(hug_trans=4.33.2)'}

In [19]:
prediction

" The STL has no concept of a \\'for\\' loop or a \\'while\\' loop, or of any sort of flow control at all (except `switch']"

In [20]:
a

["Without seeing your code it's hard to make specific comments about how to speed up what you are doing. However, `vector::begin()` is used to return an iterator to the first element in a vector - it's a standard routine when iterating across a vector.\n\nI'd actually recommend using a more modern profiler such as [OProfile](http://oprofile.sourceforge.net/docs/), this will give you much finer-grained information on where your program is spending it's time - down to the actual C++ line, or even the individual asm instruction, depending on how you run it.\n\nAs an aside - why did you choose to use [bitarray.cpp](http://michael.dipperstein.com/bitlibs/) instead of a vanilla `std::vector<bool>`? I've not used it myself, but a quick scan of the above link suggests that `bitarray.cpp` supports additional functionality over `std::vector<bool>`, which if you arn't making use of may well add overhead compared to the STL vector class..."]

In [18]:
rouge.compute(predictions=[prediction], references=a)

{'rouge1': 0.0851063829787234,
 'rouge2': 0.02150537634408602,
 'rougeL': 0.06382978723404255,
 'rougeLsum': 0.07446808510638299}

In [10]:
rouge.compute(predictions=[prediction], references=a)

{'rouge1': 0.19619619619619622,
 'rouge2': 0.03410230692076229,
 'rougeL': 0.07207207207207207,
 'rougeLsum': 0.11011011011011011}

In [4]:
for example in dataset:
    q = example["question"]
    a = example["response_j"]
    print('Evaluating question: ',  f'{q[:128]}', '...')
    sequences = prediction = pipeline(
        f'### Question: {q}',
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512
    )
    prediction = str([s['generated_text'] for s in sequences])
    metric = rouge.compute(a, prediction)
    rouge_1.append[metric['rouge1']]
    rouge_2.append[metric['rouge2']]
    print('Rouge 1 mean so far:', mean(rouge_1))
    print('Rouge 2 mean so far:', mean(rouge_2))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Evaluating question:  I'm using Dipperstein's bitarray.cpp class to work on bi-level (black and white) images where the image data is natively stored  ...


TypeError: EvaluationModule.compute() takes 1 positional argument but 3 were given

In [10]:
'hello there'.split(' ')

['hello', 'there']

In [5]:
rouge.compute(predictions = ['hello there mate whats up'], references=['hello thereffwe'])

{'rouge1': 0.28571428571428575,
 'rouge2': 0.0,
 'rougeL': 0.28571428571428575,
 'rougeLsum': 0.28571428571428575}